In [1]:
#현재 위치 확인
!pwd

/content


In [2]:
#ACGPN 모델 파일 다운로드
!git clone https://github.com/kairess/ACGPN.git

Cloning into 'ACGPN'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 165 (delta 21), reused 19 (delta 18), pack-reused 141
Receiving objects: 100% (165/165), 303.15 KiB | 10.45 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [3]:
#다운로드 받은 파일의 위치로 이동
%cd ACGPN

/content/ACGPN


dependencies 및 사전 파일 준비

In [4]:
!pip install -U --no-cache-dir gdown --pre -qq
!pip install ninja -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 6.5 MB/s eta 0:00:00


In [5]:
import gdown
import numpy as np
from PIL import Image
import IPython
import os
import sys
import time

from predict_pose import generate_pose_keypoints

In [6]:
#모델을 돌리기 위한 파일 준비
!mkdir Data_preprocessing/test_color
!mkdir Data_preprocessing/test_colormask
!mkdir Data_preprocessing/test_edge
!mkdir Data_preprocessing/test_img
!mkdir Data_preprocessing/test_label
!mkdir Data_preprocessing/test_mask
!mkdir Data_preprocessing/test_pose
!mkdir inputs
!mkdir inputs/img
!mkdir inputs/cloth

In [7]:
#현재 위치 확인
!pwd

/content/ACGPN


사전학습된 모델 다운로드

In [8]:
#ACGPN 모델에 필요한 파일 생성
!mkdir checkpoints

#사전 학습된 ACGPN 모델 다운로드
gdown.download('https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx', output='checkpoints/ACGPN_checkpoints.zip', quiet=False)

#압축된 모델 파일 풀어서 전에 만든 파일위치에 넣어줌
!unzip checkpoints/ACGPN_checkpoints.zip -d checkpoints

Downloading...
From (uriginal): https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx
From (redirected): https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx&confirm=t&uuid=ef2936c4-27da-423b-90f0-9818f57df393
To: /content/ACGPN/checkpoints/ACGPN_checkpoints.zip
100%|██████████| 524M/524M [00:07<00:00, 71.4MB/s]


Archive:  checkpoints/ACGPN_checkpoints.zip
   creating: checkpoints/label2city/
  inflating: checkpoints/label2city/latest_net_G.pth  
  inflating: checkpoints/label2city/latest_net_G1.pth  
  inflating: checkpoints/label2city/latest_net_G2.pth  
  inflating: checkpoints/label2city/latest_net_U.pth  
  inflating: checkpoints/label2city/opt.txt  


In [13]:
#옷 데이터셋이 올바른 위치에 있는지 확인
sorted(os.listdir('inputs/cloth'))

['000010_1.jpg']

In [14]:
#전처리 할 옷 데이터 이름을 시간으로 지정 - U-2-Net모델과 분리하여 테스트함으로써 필요 (실제 데이터는 런타임 도중 직접 파일에 넣어줌)
cloth_name = f'cloth_{int(time.time())}.png'

#전처리 할 옷 데이터의 파일 위치 확인
cloth_path = os.path.join('inputs/cloth', sorted(os.listdir('inputs/cloth'))[0])

#이미지 할당
cloth = Image.open(cloth_path)

#전처리를 위해 옷 이미지 파일을 해당 파일 위치에 지정해준 옷 데이터 이름으로 저장
cloth.save(os.path.join('Data_preprocessing/test_color', cloth_name))

In [10]:
#모델 데이터셋이 올바른 위치에 있는지 확인
sorted(os.listdir('inputs/img'))

['000020_0.jpg']

In [11]:
#전처리 할 모델 데이터 이름을 시간으로 지정
img_name = f'img_{int(time.time())}.png'

#전처리 할 모델 데이터의 파일 위치 확인
img_path = os.path.join('inputs/img', sorted(os.listdir('inputs/img'))[0])

#이미지 할당
img = Image.open(img_path)

#지정해준 모델 이미지 파일의 위치값을 바탕으로 이미지 저장
img.save(os.path.join('Data_preprocessing/test_img', img_name))

In [16]:
#ACGPN 추론 실행
!python test.py

?
------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
continue_train: False
data_type: 32
datapairs: test_pairs.txt
dataroot: Data_preprocessing/
display_winsize: 512
engine: None
export_onnx: None
fineSize: 512
gpu_ids: [0]
how_many: 1000
input_nc: 3
isTrain: False
label_nc: 20
loadSize: 512
max_dataset_size: inf
model: pix2pixHD
nThreads: 1
n_blocks_global: 4
n_blocks_local: 3
n_downsample_global: 4
n_local_enhancers: 1
name: label2city
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: scale_width
results_dir: ./results/
serial_batches: True
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: latest
-------------- End ----------------
CustomDatasetDataLoader
dataset [AlignedDataset] was created
Data_preprocessing/test_label label
Data_preprocessing/test_img img
Data_preprocessing/tes

경로 문제 때문에 오류 발생
FileNotFoundError는 코랩 코드상 문제가 아니라 모델 안에서 경로 지정하는 코드가 있음
(U-2-Net, 휴먼 세그멘테이션)

In [10]:
#경로 문제 때문에 ACGPN모델의 추론 파트가 제대로 돌아가지 않음
#모델 합쳐서 다시 테스트 해볼 예정
Image.open(f'results/test/try-on/')